# Python Programming for Chemists: Visualisation & Simulation Examples 

You will need the to install the Python libraries [ase](https://wiki.fysik.dtu.dk/ase/) and [nglview](https://github.com/nglviewer/nglview)

e.g.: `pip install ase nglview`

**Note**: Its not expected that you understand in detail what is happening in this notebook yet. Its merely a demonstration of what can be done with a few lines of Python code.  
However, you will understand it at the end of the lecture!


In [ ]:
!pip install ase nglview

###  Molecular Dynamics Simulation of Acetonitrile


A [molecular dynamics](https://en.wikipedia.org/wiki/Molecular_dynamics#:~:text=Molecular%20dynamics%20(MD)%20is%20a,%22evolution%22%20of%20the%20system.) (MD) simulates atomic and molecular movements over time by solving Newton's equations of motion, using force fields to calculate interactions.   
It is widely used in chemical physics, materials science, and biophysics.


In [3]:
# load the necessary functions and modules
import ase.units as units
import numpy as np
from ase.calculators.acn import ACN
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory,write, read
from ase.md import Langevin
from ase.visualize import view

# helper functions for showing output of simulation
def show_system(atoms):
    v = view(atoms, viewer="ngl")
    v.view._remote_call("setSize", target="Widget", args=["700px","700px"])
    v.view.add_ball_and_stick()
    v.view.background="black"
    return v

def print_energy(atoms):
    epot = atoms.get_potential_energy()
    ekin = atoms.get_kinetic_energy()
    temperature = atoms.get_kinetic_energy()/len(atoms)/(1.5*units.kB)
    print(f"Potential Energy: {epot:.3f} Kinetic Energy: {ekin:.3f} Temperature: {temperature:.1f}")


In [1]:
!cat ..\data\acn.xyz

81
Lattice="13.336197268014928 0.0 0.0 0.0 13.336197268014928 0.0 0.0 0.0 13.336197268014928" Properties=species:S:1:pos:R:3:masses:R:1:momenta:R:3:forces:R:3 energy=3.7601493632643694 pbc="T T T"
C        2.43932531       3.22931195       1.34129191      15.03500000       0.93334670       2.09655093       2.14097371       0.12475497      -0.62269410      -0.12934617
C        2.29165175       2.15949701       2.32080838      12.01100000       0.18147401      -0.09897079      -0.31205941       0.00852750       0.16136421      -0.01730103
N        2.17446497       1.31054235       3.09810642      14.00700000      -0.31044581      -1.75697678      -2.23551601      -0.05473280       0.52840441       0.24027051
C        2.18677718       3.05295914       5.99073967      15.03500000      -0.06352891       1.64647993       3.26064309       0.02677035      -0.51936278      -0.13014682
C        2.23567630       1.94270219       6.93450998      12.01100000       0.09210460      -0.64798002       

In [4]:
# read coorrdinates from file
atoms = read("../data/acn.xyz") 
tag = 'acn'

# set up some constraints, to accelerate dynamics
atoms.constraints = FixLinearTriatomic(
    triples=[(3 * i, 3 * i + 1, 3 * i + 2)
             for i in range(27)])

In [5]:
%%time
# define the molecular dynamics physics / rules / model
atoms.calc = ACN(rc=6.6)
# define the simulation details, temperature, time steps, parameters, logfile name
md = Langevin(atoms, 1 * units.fs,
              temperature_K=300,
              friction=0.01,
              logfile=tag + '.log')

# define a file to save the generated structures during the dynamics
traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=10)
md.attach(print_energy,interval=10,atoms=atoms)

# start the simulation for n time steps
n_steps = 500
md.run(n_steps)

Potential Energy: 3.908 Kinetic Energy: 21.515 Temperature: 2054.9
Potential Energy: 17.531 Kinetic Energy: 7.633 Temperature: 729.1
Potential Energy: 10.519 Kinetic Energy: 14.570 Temperature: 1391.5
Potential Energy: 1.284 Kinetic Energy: 23.280 Temperature: 2223.5
Potential Energy: 1.623 Kinetic Energy: 22.687 Temperature: 2166.8
Potential Energy: 4.287 Kinetic Energy: 20.011 Temperature: 1911.2
Potential Energy: 3.599 Kinetic Energy: 20.393 Temperature: 1947.7
Potential Energy: 3.928 Kinetic Energy: 19.708 Temperature: 1882.3
Potential Energy: 4.521 Kinetic Energy: 18.521 Temperature: 1769.0
Potential Energy: 3.534 Kinetic Energy: 19.289 Temperature: 1842.3
Potential Energy: 2.732 Kinetic Energy: 19.542 Temperature: 1866.5
Potential Energy: 2.289 Kinetic Energy: 19.698 Temperature: 1881.3
Potential Energy: 3.373 Kinetic Energy: 18.235 Temperature: 1741.7
Potential Energy: 2.942 Kinetic Energy: 18.175 Temperature: 1735.9
Potential Energy: 2.324 Kinetic Energy: 18.264 Temperature: 17

True

In [6]:
# visualization of the dynamics
traj = Trajectory("acn.traj")
show_system(traj)

### Exercise

* Find the temperature parameter. Set the temperature to 100 K and observe the result.
* The result will not be what you expect. Which parameter should be changed in addition?
* How would you change the code to continue on the previous results?